# Retrieval Augmented Generation (RAG) - Synthesis

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Runnable

#### Runnable Lambda

In [2]:
from langchain_core.runnables import RunnableLambda

# Define a RunnableLambda that takes a string as input,
# reverses it, and then repeats the reversed string twice.
runnable = RunnableLambda(lambda s: (s[::-1]) * 2)

# Invoke the runnable with the string "hello"
result = runnable.invoke("hello")

print(result)  # Output: "olleholleh"

olleholleh


#### Runnable Sequence

Implementation using the | operator

In [3]:
from langchain_core.runnables import RunnableLambda

# Define a sequence of RunnableLambda instances that process a string
# Step 1: Remove leading and trailing whitespaces
# Step 2: Convert the string to uppercase
# Step 3: Replace spaces with underscores
# Step 4: Add "Processed:" prefix to the string
sequence = RunnableLambda(lambda s: s.strip()) | \
           RunnableLambda(lambda s: s.upper()) | \
           RunnableLambda(lambda s: s.replace(" ", "_")) | \
           RunnableLambda(lambda s: f"Processed: {s}")    

# Invoke the sequence with the input string "  hello world  "
result = sequence.invoke("  hello world  ")

print(result)  # Output: "Processed: HELLO_WORLD"

Processed: HELLO_WORLD


Alternative implementation by directly calling the RunnableSequence

In [4]:
from langchain_core.runnables import RunnableLambda, RunnableSequence

# Define individual RunnableLambda steps for processing a string
strip = RunnableLambda(lambda s: s.strip())             # Step 1: Remove leading and trailing whitespaces
uppercase = RunnableLambda(lambda s: s.upper())         # Step 2: Convert the string to uppercase
replace = RunnableLambda(lambda s: s.replace(" ", "_")) # Step 3: Replace spaces with underscores
prefix = RunnableLambda(lambda s: f"Processed: {s}")    # Step 4: Add "Processed:" prefix to the string

# Create a RunnableSequence that combines all the individual steps
sequence = RunnableSequence(strip, uppercase, replace, prefix)

# Invoke the sequence with the input string "  hello world  "
result = sequence.invoke("  hello world  ")

print(result)  # Output: "Processed: HELLO_WORLD"

Processed: HELLO_WORLD


#### Runnable Parallel 

In [5]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

# Define individual RunnableLambda functions for different string operations
reverse = RunnableLambda(lambda s: s[::-1])      # Function to reverse the input string
uppercase = RunnableLambda(lambda s: s.upper())  # Function to convert the string to uppercase
length = RunnableLambda(lambda s: len(s))        # Function to calculate the length of the string

# Create a RunnableParallel with a mapping (dictionary) of operations
parallel = RunnableParallel({
    "reverse": reverse,
    "uppercase": uppercase,
    "length": length
})

# Invoke the parallel sequence with the input string "hello"
result = parallel.invoke("hello")

print(result)  # Output: {'reverse': 'olleh', 'uppercase': 'HELLO', 'length': 5}

{'reverse': 'olleh', 'uppercase': 'HELLO', 'length': 5}


#### Runnable Passthrough

In [6]:
from langchain_core.runnables import RunnablePassthrough

# Define a RunnablePassthrough that passes the input unchanged
passthrough = RunnablePassthrough()

# Invoke the passthrough
result = passthrough.invoke("hello world")

print(result)  # Output: "hello world"

hello world


## Prompt Templates
#### Basic Prompt Template

In [7]:
from langchain_core.prompts import PromptTemplate

# Define a template for the prompt that includes a placeholder for the country
template = "What is the capital of {country}?"

# Create a PromptTemplate instance with the defined template
prompt_template = PromptTemplate(template=template)

# Format the template by replacing the placeholder with the specified country ("Pakistan")
prompt = prompt_template.format(country="Pakistan")

print(prompt)  # Output: What is the capital of Pakistan?

What is the capital of Pakistan?


#### Few-Shot Prompt Template

In [8]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

# Define a list of example queries and their corresponding answers
examples = [
    {"query": "How can I improve my time management skills?", 
     "answer": "Start by prioritizing your tasks using the Eisenhower Matrix to distinguish between what's urgent and important."},
     
    {"query": "What are some effective strategies for setting goals?", 
     "answer": "Use the SMART criteria: make your goals Specific, Measurable, Achievable, Relevant, and Time-bound."}
]

# Create a PromptTemplate for formatting the examples
example_template = PromptTemplate(
    input_variables=["query", "answer"],  # Define the input variables for the template
    template="User: {query}\nAI: {answer}"  # Template structure for each example
)

# Create a FewShotPromptTemplate using the examples and example_template
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,                                        
    example_prompt=example_template,                           
    prefix="Here are some examples of questions and answers:",  
    suffix="User: {query}\nAI: {answer}\n", 
    input_variables=["query"],              
    example_separator="\n\n"
)

# Format the few-shot prompt with a new user query and corresponding answer
prompt = few_shot_prompt.format(query="How do I stay motivated during long projects?",
                                answer="Break down the project into smaller milestones and celebrate each achievement.")

print(prompt)

Here are some examples of questions and answers:

User: How can I improve my time management skills?
AI: Start by prioritizing your tasks using the Eisenhower Matrix to distinguish between what's urgent and important.

User: What are some effective strategies for setting goals?
AI: Use the SMART criteria: make your goals Specific, Measurable, Achievable, Relevant, and Time-bound.

User: How do I stay motivated during long projects?
AI: Break down the project into smaller milestones and celebrate each achievement.



#### Chat Prompt Template

In [9]:
from langchain_core.prompts import ChatPromptTemplate

# Define a ChatPromptTemplate from a list of message tuples. 
# The first element in each tuple represents the speaker ("human" or "ai"), 
# and the second element is the message template.
chat_template = ChatPromptTemplate.from_messages([
    ("human", "Can you tell me about {author}?"),   # Human asks about the author
    ("ai", "{author} is a renowned author known for {notable_work}.")  # AI responds with author details
])

# Format the chat prompt with specific values for the placeholders
# Replaces {author} with "Jane Austen" and {notable_work} with "Pride and Prejudice"
messages = chat_template.format_messages(author="Jane Austen", notable_work="Pride and Prejudice")

print(messages)

[HumanMessage(content='Can you tell me about Jane Austen?', additional_kwargs={}, response_metadata={}), AIMessage(content='Jane Austen is a renowned author known for Pride and Prejudice.', additional_kwargs={}, response_metadata={})]


## Output Parsers

#### StrOutputParser

In [10]:
from langchain_core.output_parsers import StrOutputParser

# Initialize the parser
parser = StrOutputParser()

# Define a string output from an AI explaining a concept
ai_response = """
Machine learning is a subset of artificial intelligence that focuses on enabling systems to learn from data patterns and make decisions without being explicitly programmed. It is widely used in applications like recommendation systems, image recognition, and natural language processing.
"""

# Use the parser to process the AI response
parsed_output = parser.invoke(ai_response)

print(parsed_output)


Machine learning is a subset of artificial intelligence that focuses on enabling systems to learn from data patterns and make decisions without being explicitly programmed. It is widely used in applications like recommendation systems, image recognition, and natural language processing.



#### JSONOutputParser

In [11]:
from langchain_core.output_parsers import JsonOutputParser

# Initialize the JSON parser
parser = JsonOutputParser()

# Simulated AI response in JSON format (string)
ai_response = """
{
    "name": "John Doe",
    "age": 30,
    "occupation": "Software Engineer",
    "skills": ["Python", "Machine Learning", "Data Analysis"]
}
"""

# Parse the JSON response
parsed_output = parser.invoke(ai_response)

# Print the parsed output as a dictionary
print(parsed_output)

# Access specific fields from the parsed JSON output
print(f"Name: {parsed_output['name']}")
print(f"Skills: {', '.join(parsed_output['skills'])}")

{'name': 'John Doe', 'age': 30, 'occupation': 'Software Engineer', 'skills': ['Python', 'Machine Learning', 'Data Analysis']}
Name: John Doe
Skills: Python, Machine Learning, Data Analysis


### Data Ingestion & Retrieval
Let's ingest and retrieve data before calling chains for synthesis.

In [12]:
# Data Ingestion & Retrieval
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# Load a PDF document and split it into chunks
file_path = "data/Artificial Intelligence.txt"  # Path of the document to be loaded
loader = TextLoader(file_path)                  # Initialize the text loader
documents = loader.load()                       # Load the txt document 

# Initialize the recursive character text splitter
text_splitter = RecursiveCharacterTextSplitter(              
    separators="",
    chunk_size=100,
    chunk_overlap=20
)   

# Split the documents into chunks
chunks = text_splitter.split_documents(documents)

# Initialize the Hugging Face embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Store embeddings into the vector store
vector_store = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)

# Retrieve relevant information using similarity search
retriever = vector_store.as_retriever() # uses similarity search by default

Load the `Groq API key` using the system environment variable.

In [13]:
import os
from dotenv import load_dotenv

# Set the environment variable for the Groq API key using the system environment variable
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

## **Chains**
#### 1. Question/Answering (Q/A) Chain

In [16]:
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq

# Define a prompt template for formatting the question
prompt_template = "Q: {question}\n"
prompt = PromptTemplate(template=prompt_template, input_variables=["question"])

# Instantiate the ChatGroq LLM with the specified model
llm = ChatGroq(model="mixtral-8x7b-32768")

# Combine the prompt and LLM into a chain where the prompt output is passed to the LLM
qa_chain = prompt | llm  # This creates a chain that takes the formatted question and passes it to the model

# Invoke the chain with a specific question
response = qa_chain.invoke("What is the capital of Japan?")

# Print the content of the response
print(response.content)

The capital of Japan is Tokyo. Established in 1869, Tokyo is one of the 47 prefectures of Japan. It is located on the eastern coast of the island of Honshu and is the most populous city in the world. Tokyo is known for its bustling city life, rich history, and cultural landmarks such as the Tokyo Tower, the Tokyo Skytree, and the historic Asakusa district. It is also home to the Japanese government and the Imperial Palace.


#### 2. Question/Answering (Q/A) Retrieval Chain

In [17]:
from langchain_groq import ChatGroq
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Initialize the ChatGroq LLM with the specific model
llm = ChatGroq(model="mixtral-8x7b-32768")

# Define the system prompt that instructs the LLM how to answer questions based on retrieved context
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"  # Placeholder for the retrieved context
)

# Create a chat prompt template with a system message and human message
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),  # System message contains instructions and context
        ("human", "{input}"),  # Human message includes the user's input question
    ]
)

# Create a document chain that combines the LLM with the prompt
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Combine the retrieval chain with the question-answering chain
# The retrieval chain retrieves relevant documents and feeds them into the question-answering chain
retrieval_chain = create_retrieval_chain(retriever, question_answer_chain)

# Invoke the chain with a question, and the retriever will provide context for the LLM to generate an answer
response = retrieval_chain.invoke({"input": "What is Artificial Intelligence?"})

print(response['answer'])

Artificial Intelligence (AI) refers to intelligence demonstrated by machines. It is often defined as the ability to solve hard problems or the study of how to make computers do things at which, at the moment, people are better. AI agents are software entities that perceive their environment.


#### 3. Conversational Chain

In [18]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_groq import ChatGroq

# Create an external store (a dictionary) for maintaining chat history across multiple sessions
store = {}

# Function to get or create chat history for a specific session based on session_id
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    # If the session_id doesn't exist in the store, create a new InMemoryChatMessageHistory
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    # Return the chat history associated with the given session_id
    return store[session_id]

# Initialize the ChatGroq model
llm = ChatGroq(model="mixtral-8x7b-32768")

# Create a chain that can handle message history, linking it with the LLM and session history retrieval
chain = RunnableWithMessageHistory(llm, get_session_history)

# First conversation with AI in session "1"
question_1 = "Hello, there?"
response_1 = chain.invoke(
    question_1,
    config={"configurable": {"session_id": "1"}},  # Specify the session_id for tracking message history
)

# Second conversation in the same session "1"
question_2 = "What can you help me with today?"
response_2 = chain.invoke(
    question_2,
    config={"configurable": {"session_id": "1"}},  # Continue in the same session "1"
)

# Third conversation in the same session "1"
question_3 = "Can you explain the concept of AI in one sentence?"
response_3 = chain.invoke(
    question_3,
    config={"configurable": {"session_id": "1"}},  # Continue in session "1"
)

# Print the responses to see the conversation flow
print("Human:", question_1)
print("AI:   ", response_1.content, "\n")

print("Human:", question_2)
print("AI:   ", response_2.content, "\n")

print("Human:", question_3)
print("AI:   ", response_3.content, "\n")

Human: Hello, there?
AI:    Hello! Yes, I'm here. How can I help you today? If you have any questions about computer programming or software development, feel free to ask. I'll do my best to provide a clear and helpful answer. 😊 

Human: What can you help me with today?
AI:    I can help you with a variety of topics related to computer programming and software development. Here are a few examples:

1. Explaining programming concepts: If you're new to programming or need help understanding a specific concept, I can explain it in simple terms.
2. Debugging code: If you're having trouble with a piece of code, I can help you identify and fix issues.
3. Providing code examples: If you're looking for a code example to learn from or use in your own projects, I can provide that.
4. Offering guidance on best practices: If you're unsure about the best way to approach a programming problem or design a software system, I can offer advice based on my experience.
5. Reviewing and improving code: If 

#### 4. Conversational Retrieval Chain

In [19]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_groq import ChatGroq

# Initialize the large language model
llm = ChatGroq(model="mixtral-8x7b-32768")

### Contextualize question ###
# Define a system prompt to contextualize the user question by making it standalone
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

# Create a ChatPromptTemplate for contextualizing questions with chat history
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),      
        ("human", "{input}"),                      
    ]
)

# Create a history-aware retriever to handle chat history
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

### Answer question ###
# Define a system prompt to generate concise answers using retrieved context
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"  # Placeholder for retrieved context
)

# Create a ChatPromptTemplate for the question-answering chain
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),            
        MessagesPlaceholder("chat_history"),   
        ("human", "{input}"),                
    ]
)

# Create a document chain for handling question answering using the LLM and the QA prompt
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Combine the history-aware retriever and the QA chain to create the retrieval chain
retrieval_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

### Statefully manage chat history ###
# Store for maintaining chat history across multiple sessions
store = {}

# Function to get or create chat history for a given session
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    # If no session history exists for the given session_id, create a new history
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Create a conversational retrieval chain that manages chat history statefully
conversational_retrieval_chain = RunnableWithMessageHistory(
    retrieval_chain,                 
    get_session_history,           
    input_messages_key="input",     
    history_messages_key="chat_history",
    output_messages_key="answer",
)

# First conversation: question about the document
question_1 = {"input": "What is the document about?"}
response_1 = conversational_retrieval_chain.invoke(
    question_1,
    config={
        "configurable": {"session_id": "abc123"}
    },
)["answer"]

# Second conversation in the same session: asking to summarize the main points
question_2 = {"input": "Summarize its main points?"}
response_2 = conversational_retrieval_chain.invoke(
    question_2,
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

# Printing the responses to see the conversational retrieval flow
print("Human:", question_1["input"])
print("AI:   ", response_1, "\n")

print("Human:", question_2["input"])
print("AI:   ", response_2, "\n")

Human: What is the document about?
AI:    The document is about developing artificial intelligence (AI) in accordance with human rights and democratic values, prioritizing user needs over speculative scenarios, and addressing concerns about fulfilling these principles regardless of the source. A ChatGPT search presumably relates to this context, but the specifics are not provided. 

Human: Summarize its main points?
AI:    The main points of the document are:

1. AI development should prioritize user needs, human rights, and democratic values.
2. AI should not infringe upon human rights, and granting rights to AI systems could undermine their importance.
3. The UK is discussing near and far-term AI risks, considering mandatory and voluntary regulation to address these concerns. 



## **Example - Q/A Retrieval Chain**

#### **Data Ingestion**
##### Load, Split, Embed, and Store a PDF Document

In [20]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# Load a PDF document and split it into chunks
file_path = "data/Dale_Carnegie_Golden_Book-Se.pdf"  # Path of the document to be loaded
loader = PyPDFLoader(file_path)     # Initialize the pdf loader
documents = loader.load()           # Load the pdf document 

# Initialize the recursive character text splitter
text_splitter = RecursiveCharacterTextSplitter(              
    separators="",
    chunk_size=100,
    chunk_overlap=20
)   

# Split the documents into chunks
chunks = text_splitter.split_documents(documents)

# Initialize the Hugging Face embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Store embeddings into the vector store
vector_store = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)

#### **Data Retrieval**
##### Retrieve Documents using Vector Store as a Retriever

In [21]:
# Retrieve relevant information using similarity search
retriever = vector_store.as_retriever() # uses similarity search by default

#### **Synthesis**
##### Generate Response using a Q/A retrieval chain

In [22]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Set the environment variable for the Groq API key using the system environment variable
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the prompt template to structure how the input context and question are fed into the model
template = """
Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate(template=template)

# Initialize the LLM (Groq)
llm = ChatGroq(model="mixtral-8x7b-32768")

# Define the chain of operations:
# 1. Get the context through the retriever.
# 2. Pass the question without any change using RunnablePassthrough.
# 3. Format the input (context & question) through PromptTemplate
# 4. Process the prompt through an LLM and return a response.
# 5. Parse the response as a string output by StrOutputParser()
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()} # Step 1: Pass the context and question
    | prompt                                                                # Step 2: Format them with the prompt template
    | llm                                                                   # Step 3: Feed the prompt into the model (Groq)
    | StrOutputParser()                                                     # Step 4: Parse the model's response into a string
)

# Invoke the chain to answer the question
response = chain.invoke("What are the qualities of a good leader?")
print(response)

Based on the provided context, a good leader should possess the following qualities:

1. Begin with praise and honest appreciation: A good leader should start by acknowledging the strengths and achievements of others.
2. Show tolerance and avoid seeking revenge: A good leader should not hold grudges or try to get even with their enemies, instead, they should show understanding and forgiveness.
3. Be prepared for ingratitude: A good leader should not expect constant appreciation or recognition, and should be able to handle ingratitude with grace.
4. Strive for excellence: A good leader should always aim to do their best in every situation, and continuously work towards improvement.
5. Self-analyze and self-criticize: A good leader should have the ability to objectively analyze their own mistakes, and be willing to accept criticism and learn from it.
